## Preparations
* TensorFlow version == 1.x required
* Gast version == 0.2.x, TensorFlow-Gan version == 1.x required

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import io_ops
from scipy.stats import norm
import matplotlib.pyplot as plt
from tensorflow.python.ops import io_ops
tfd = tf.contrib.distributions

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import utils
import nets
import flow_layers as fl

In [ ]:
print(tf.__version__)

In [ ]:
config = tf.ConfigProto()
config.allow_soft_placement=True
config.gpu_options.allow_growth = True
tf.set_random_seed(0)
sess = tf.InteractiveSession(config=config)

## Main Part

In [ ]:
batch_size = 1
image_size = 128

In [ ]:
nn_template_fn = nets.OpenAITemplate(
    width=128
)

In [ ]:
layers_encoder, actnorm_layers_encoder = nets.create_simple_flow(
    num_steps=32, 
    num_scales=5, 
    template_fn=nn_template_fn
)

In [ ]:
for i in range(1,3):
    tensor_reader = tf.train.NewCheckpointReader("aux-saves/" + str(i) + ".ckpt")
    tmp = tensor_reader.get_tensor(str(i))
    image = tf.convert_to_tensor(tmp)
    flow_encoder = fl.InputLayer(tf.expand_dims(image, axis=0))
    a, _, _ = flow_encoder
    save_op = io_ops._save(filename="flow_encoder/a" + str(i) + ".ckpt", tensor_names=["a" + str(i)],
                       tensors=[a])
    sess.run(save_op) 

In [ ]:
for i in range(1,3):
    tensor_reader = tf.train.NewCheckpointReader("aux-saves/" + str(i) + ".ckpt")
    tmp = tensor_reader.get_tensor(str(i))
    image = tf.convert_to_tensor(tmp)
    
    flow_encoder = fl.InputLayer(tf.expand_dims(image, axis=0))
    _, b, _ = flow_encoder
    save_op = io_ops._save(filename="flow_encoder/b" + str(i) + ".ckpt", tensor_names=["b" + str(i)],
                       tensors=[b])
    sess.run(save_op) 

In [ ]:
image_hold = tf.placeholder(tf.float32, [1, image_size, image_size, 3])

In [ ]:
flow = fl.InputLayer(image_hold)
model_flow = fl.ChainLayer(layers_encoder)
output_flow = model_flow(flow, forward=True)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, "t1-1-saves/steps.ckpt")

In [ ]:
for i in range(1,3):
    reader_a = tf.train.NewCheckpointReader("flow_encoder/a" + str(i) + ".ckpt")
    tmp_a = reader_a.get_tensor("a" + str(i))
    x = tf.convert_to_tensor(tmp_a)
    
    reader_b = tf.train.NewCheckpointReader("flow_encoder/b" + str(i) + ".ckpt")
    tmp_b = reader_b.get_tensor("b" + str(i))
    y = tf.convert_to_tensor(tmp_b)
    
    inputs = x, y, None
    encoder_outputs = model_flow(inputs, forward=True)
    aa, bb, cc = encoder_outputs
    
    save_op_1 = io_ops._save(filename="aux-saves/a" + str(i) + ".ckpt", tensor_names=["a" + str(i)],
                           tensors=[aa])
    save_op_2 = io_ops._save(filename="aux-saves/b" + str(i) + ".ckpt", tensor_names=["b" + str(i)],
                           tensors=[bb])
    save_op_3 = io_ops._save(filename="aux-saves/c" + str(i) + ".ckpt", tensor_names=["c" + str(i)],
                           tensors=[cc])
    
    sess.run(save_op_1) 
    sess.run(save_op_2) 
    sess.run(save_op_3) 